In [1]:
import warnings
import time
import sys
import os
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from model_zoo import my_lgb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline

In [2]:
# 评分函数与提交函数
def score(pre, truth):
    return 1 / (MAE(pre, truth) + 1)

def MAE(pre, truth):
    return abs((np.rint(pre) - truth)).mean()

def submit(model_name='default', predictions=None):
    sub_df = pd.read_csv('dataset/submit_example.csv')
    sub_df[' score'] = np.rint(predictions).astype(int)
    sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [3]:
def merge(sub1, sub2):
    pre = 0.5*sub1 + 0.5*sub2
    submit(model_name='merge', predictions=pre)

In [16]:
sub1 = pd.read_csv('output/my_model.csv')
sub2 = pd.read_csv('ImageOfConsumers-master/0216_2143result.csv')

In [21]:
res1 = sub1[' score'].values
res2 = sub2['score'].values

In [22]:
merge(res1, res2)

In [23]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

特征工程，简单粗暴。

* 长尾数据处理

* 过拟合

In [24]:
def percent(df, feature):
    for f in feature:
        data[f] = np.rint(data[f])
        new_feature = f + '_count'
        tmp = df.groupby(f).size().reset_index().rename(columns={0:new_feature})
        #  tmp[new_feature] = tmp[new_feature].cumsum()*100/df.shape[0]
        df = df.merge(tmp, 'left', on = f)
    return df
    
def generate_feature(df):
    df['用户前五个月平均消费值（元）'] = (df['用户近6个月平均消费值（元）']*6 - df['用户账单当月总费用（元）'])/5
    df['当月消费值较前五个月平均消费值'] = df['用户账单当月总费用（元）'] - df['用户前五个月平均消费值（元）']
    app_col = ['当月视频播放类应用使用次数','当月金融理财类应用使用总次数','当月网购类应用使用次数']
    df['是否使用网购类应用'] = np.where(df['当月网购类应用使用次数'] > 0, 1, 0)
    df['当月网购类应用使用次数' + '百分比'] = (df['当月网购类应用使用次数'])/(df[app_col].sum(axis=1) + 1e-8)
    df.loc[df['用户年龄']==0, '用户年龄'] = df['用户年龄'].mode()
    df['当月视频播放类应用使用次数'] = np.where(df['当月视频播放类应用使用次数']>30000, 30000, df['当月视频播放类应用使用次数'])
    df['当月网购类应用使用次数'] = np.where(df['当月网购类应用使用次数']>10000, 10000, df['当月网购类应用使用次数'])
    df['当月金融理财类应用使用总次数'] = np.where(df['当月金融理财类应用使用总次数']>10000, 10000, df['当月金融理财类应用使用总次数'])
    df['用户当月账户余额（元）'] = np.where(df['用户当月账户余额（元）']>2000, 2000, df['用户当月账户余额（元）'])
    return df

In [25]:
target = train_df['信用分']
data = pd.concat([train_df.drop(columns=['信用分']), test_df], axis=0, ignore_index=True)
data = generate_feature(data)
train = data.loc[:49999, :]
test = data.loc[50000:, :]

In [26]:
drop_columns = ['用户编码']
X_train = train.drop(columns=drop_columns).values
y_train = target.values
X_test = test.drop(columns=drop_columns).values

param = {'num_leaves': 30,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.008,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.04,
         "verbosity": -1}

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.004,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 0.15,
         "lambda_l2": 0.05,
         "verbosity": -1}
 94

阴差阳错我用的都是mse,只是用了不同的随机种子

In [30]:
param = {'num_leaves': 40,
         'objective':'regression_l2',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}
clf = my_lgb(folds=5, seed=2018)
clf.inference_folds(X_train, y_train, X_test, param)
mse_2018 = clf.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.5586	valid_1's l1: 20.8048
[400]	training's l1: 16.3266	valid_1's l1: 16.7959
[600]	training's l1: 14.9926	valid_1's l1: 15.6126
[800]	training's l1: 14.5343	valid_1's l1: 15.243
[1000]	training's l1: 14.3206	valid_1's l1: 15.105
[1200]	training's l1: 14.1827	valid_1's l1: 15.0438
[1400]	training's l1: 14.0768	valid_1's l1: 15.0121
[1600]	training's l1: 13.9903	valid_1's l1: 14.9939
[1800]	training's l1: 13.9107	valid_1's l1: 14.9788
[2000]	training's l1: 13.8408	valid_1's l1: 14.9678
[2200]	training's l1: 13.7702	valid_1's l1: 14.9588
[2400]	training's l1: 13.7073	valid_1's l1: 14.9533
[2600]	training's l1: 13.6471	valid_1's l1: 14.9485
[2800]	training's l1: 13.5873	valid_1's l1: 14.9456
[3000]	training's l1: 13.5295	valid_1's l1: 14.9407
[3200]	training's l1: 13.4715	valid_1's l1: 14.9376
[3400]	training's l1: 13.4182	valid_1's l1: 14.934
[3600]	training's l1: 13.3634	valid_1's l1: 14.932


In [39]:
param = {'num_leaves': 40,
         'objective':'regression_l2',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mse = my_lgb(folds=5, seed=2019)
clf_mse.inference_folds(X_train, y_train, X_test, param)
mse_2019 = clf_mse.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.6206	valid_1's l1: 20.6877
[400]	training's l1: 16.4086	valid_1's l1: 16.5454
[600]	training's l1: 15.0726	valid_1's l1: 15.3227
[800]	training's l1: 14.6043	valid_1's l1: 14.9491
[1000]	training's l1: 14.3874	valid_1's l1: 14.8153
[1200]	training's l1: 14.2527	valid_1's l1: 14.7528
[1400]	training's l1: 14.1457	valid_1's l1: 14.7169
[1600]	training's l1: 14.0588	valid_1's l1: 14.6949
[1800]	training's l1: 13.9817	valid_1's l1: 14.6793
[2000]	training's l1: 13.9112	valid_1's l1: 14.6678
[2200]	training's l1: 13.8444	valid_1's l1: 14.6595
[2400]	training's l1: 13.7823	valid_1's l1: 14.6526
[2600]	training's l1: 13.7216	valid_1's l1: 14.648
[2800]	training's l1: 13.6595	valid_1's l1: 14.6441
[3000]	training's l1: 13.602	valid_1's l1: 14.642
[3200]	training's l1: 13.546	valid_1's l1: 14.6401
[3400]	training's l1: 13.4925	valid_1's l1: 14.6374
[3600]	training's l1: 13.4375	valid_1's l1: 14.6336


In [ ]:
p = 0.502
q = 0.50
results = mse_2018*p+ mse_2019*q 
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q
# submit(model_name='model_V2_bad', predictions=res)

In [44]:
param = {'num_leaves': 40,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 1,
         "verbosity": -1}

clf_mae = my_lgb(folds=5, seed=2018)
clf_mae.inference_folds(X_train, y_train, X_test, param)
mae_2018 = clf_mae.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.5719	valid_1's l1: 20.833
[400]	training's l1: 16.5562	valid_1's l1: 17.0275
[600]	training's l1: 15.148	valid_1's l1: 15.7876
[800]	training's l1: 14.6116	valid_1's l1: 15.3602
[1000]	training's l1: 14.3555	valid_1's l1: 15.1767
[1200]	training's l1: 14.1917	valid_1's l1: 15.0894
[1400]	training's l1: 14.0666	valid_1's l1: 15.0374
[1600]	training's l1: 13.9633	valid_1's l1: 15.0058
[1800]	training's l1: 13.8701	valid_1's l1: 14.9823
[2000]	training's l1: 13.7875	valid_1's l1: 14.9659
[2200]	training's l1: 13.7052	valid_1's l1: 14.9513
[2400]	training's l1: 13.6302	valid_1's l1: 14.9411
[2600]	training's l1: 13.5587	valid_1's l1: 14.932
[2800]	training's l1: 13.4925	valid_1's l1: 14.9265
[3000]	training's l1: 13.4273	valid_1's l1: 14.9225
[3200]	training's l1: 13.3647	valid_1's l1: 14.9177
[3400]	training's l1: 13.3038	valid_1's l1: 14.9172
[3600]	training's l1: 13.2464	valid_1's l1: 14.9135

In [45]:
# 0.06398976
clf_mae.submit(output_name = 'model_V1_mae_2018_1')

In [52]:
param = {'num_leaves': 40,
         'objective':'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mae',
         "lambda_l1": 5,
         "lambda_l2": 0,
         "verbosity": -1}

clf_mae_ = my_lgb(folds=5, seed=2019)
clf_mae_.inference_folds(X_train, y_train, X_test, param)
mae_2019 = clf_mae_.oof

fold n°1
Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 20.5755	valid_1's l1: 20.6502
[400]	training's l1: 16.5486	valid_1's l1: 16.7249
[600]	training's l1: 15.1491	valid_1's l1: 15.4617
[800]	training's l1: 14.6173	valid_1's l1: 15.0348
[1000]	training's l1: 14.3536	valid_1's l1: 14.8721
[1200]	training's l1: 14.187	valid_1's l1: 14.7975
[1400]	training's l1: 14.0599	valid_1's l1: 14.7576
[1600]	training's l1: 13.9552	valid_1's l1: 14.7327
[1800]	training's l1: 13.8631	valid_1's l1: 14.7161
[2000]	training's l1: 13.7769	valid_1's l1: 14.7012
[2200]	training's l1: 13.6925	valid_1's l1: 14.6894
[2400]	training's l1: 13.6116	valid_1's l1: 14.6786
[2600]	training's l1: 13.5373	valid_1's l1: 14.673
[2800]	training's l1: 13.4661	valid_1's l1: 14.6672
[3000]	training's l1: 13.3982	valid_1's l1: 14.6631
[3200]	training's l1: 13.3328	valid_1's l1: 14.657
[3400]	training's l1: 13.27	valid_1's l1: 14.6527
[3600]	training's l1: 13.2087	valid_1's l1: 14.6484
[

In [51]:
# 0.06399214
clf_mae_.submit(output_name = 'model_V1_mae_2019_1')

param = {'num_leaves': 35,
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.5,
         "metric": 'mse',
         "lambda_l1": 0.16,
         "lambda_l2": 0.1,
         "verbosity": -1}

In [ ]:
p = 0.25
q = 0.25
m = 0.251
n = 0.25
results = mse_2018*p+ mse_2019*q + mae_2018*m + mae_2019*n
print(score(results, y_train))
res = clf.results*p + clf_mse.results*q + clf_mae.results*m + clf_mae_.results*n
submit(model_name='model_V2_bad', predictions=res)

In [ ]:
residual = mae_2018 - y_train
submit(model_name='resdiual_mae', predictions=residual)

In [ ]:
# drop_columns = drop_columns.remove('信用分')
col = train.drop(columns=drop_columns).columns
f = clf_mae.importance_feature(col)
f

In [ ]:
(residual>100).sum()

In [ ]:
feature = ['用户网龄（月）', '当月通话交往圈人数', '用户年龄', 
       '用户账单当月总费用（元）', '用户近6个月平均消费值（元）', '当月金融理财类应用使用总次数', '当月视频播放类应用使用次数',
       '当月网购类应用使用次数', '用户当月账户余额（元）', '近三个月月均商场出现次数',
       '当月旅游资讯类应用使用次数', '缴费用户最近一次缴费金额（元）', '信用分']

In [ ]:
train_df.loc[residual > 100,feature].sample(10)

In [ ]:
train_df.loc[23379, :]